<a href="https://colab.research.google.com/github/NeuroTechBSB/BR41N.IO-2025/blob/main/ecog_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Uploading the data files locally in the Google Drive space
from google.colab import files
files.upload() #this will prompt you to upload a file locally into the Google Drive (temporally for the session)

## Alternatively (if want to use a already downloaded file in Google Drive):
# from google.colab import drive
# drive.mount('/content/drive')
# !cp /content/drive/MyDrive/FILE.zip /content
# %%capture
# !unzip /content/FILE.zip -d data


In [114]:
!pip install pymatreader
from pymatreader import read_mat

data_temp = read_mat('ECoG_Handpose.mat')

In [122]:
data_temp

{'__header__': b'MATLAB 5.0 MAT-file, Platform: PCWIN64, Created on: Tue Jun 23 10:38:21 2020',
 '__version__': '1.0',
 '__globals__': [],
 'y': array([[ 0.00000000e+00,  8.33333333e-04,  1.66666667e-03, ...,
          4.22518333e+02,  4.22519167e+02,  4.22520000e+02],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
         -1.06701961e+05, -1.06699477e+05, -1.06697859e+05],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
         -8.62051094e+04, -8.62023672e+04, -8.61986562e+04],
        ...,
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
          1.53902262e-01,  1.53902262e-01,  1.53902262e-01],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
          1.61574081e-01,  1.61574081e-01,  1.61574081e-01],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
          2.34732822e-01,  2.34732822e-01,  2.34732822e-01]])}

In [115]:
import pandas as pd
import numpy as np

data = pd.DataFrame(data_temp['y']);
# data = 67 rows × 507025 columns

# Data matrix description: [channel x time]
# CH1: sample time
# CH2-61: ECoG (raw and DC-coupled; recorded from right sensorimotor cortex)
# CH62: paradigm info (0...relax, 1...fist movement, 2...peace movement, 3...open hand)
# CH63: data glove thumb
# CH64: data glove index
# CH65: data glove middle
# CH66: data glove ring
# CH67: data glove little

sample_time = data.loc[0,:]
ecog = data.loc[1:60,:]
paradigm_info = data.loc[61,:]
glove_thumb = data.loc[62,:]
glove_index = data.loc[63,:]
glove_ring = data.loc[64,:]
glove_litlle = data.loc[66,:]

data = data.transpose()
col_names = [f'CH_{i}' for i in range(1, 62)] # All from right sensorimotor cortex
col_names.extend(['paradigm_info','data_glove_thumb', 'data_glove_index','data_glove_middle', 'data_glove_ring', 'data_glove_little' ])
data.columns = col_names

relax = data[data['paradigm_info'] == 0]
fist_movement = data[data['paradigm_info'] == 1]
peace_movement = data[data['paradigm_info'] == 2]
open_hand = data[data['paradigm_info'] == 3]

In [121]:
data

,CH_1,CH_2,CH_3,CH_4,CH_5,CH_6,CH_7,CH_8,CH_9,CH_10,...,CH_58,CH_59,CH_60,CH_61,paradigm_info,data_glove_thumb,data_glove_index,data_glove_middle,data_glove_ring,data_glove_little
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.00000,0.000000,0.000000,0.000000,0.000000
1,0.000833,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.00000,0.000000,0.000000,0.000000,0.000000
2,0.001667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.00000,0.000000,0.000000,0.000000,0.000000
3,0.002500,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.00000,0.000000,0.000000,0.000000,0.000000
4,0.003333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.00000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
507020,422.516667,-106697.578125,-86204.289062,-76535.585938,-103944.351562,-59508.328125,-39716.640625,-12350.508789,46311.046875,13856.023438,...,-65758.367188,-33892.039062,-4235.358887,51480.582031,0.0,0.70303,0.178975,0.153902,0.161574,0.234733
507021,422.517500,-106698.921875,-86203.906250,-76537.109375,-103945.375000,-59508.292969,-39717.875000,-12354.244141,46318.351562,13859.705078,...,-65760.468750,-33896.398438,-4240.333984,51477.101562,0.0,0.70303,0.178975,0.153902,0.161574,0.234733
507022,422.518333,-106701.960938,-86205.109375,-76535.445312,-103945.046875,-59511.664062,-39718.042969,-12355.633789,46322.660156,13859.116211,...,-65757.734375,-33894.804688,-4240.388672,51475.777344,0.0,0.70303,0.178975,0.153902,0.161574,0.234733
507023,422.519167,-106699.476562,-86202.367188,-76532.234375,-103945.843750,-59508.625000,-39713.183594,-12354.968750,46326.210938,13862.730469,...,-65757.828125,-33896.714844,-4241.463867,51473.503906,0.0,0.70303,0.178975,0.153902,0.161574,0.234733


In [127]:
data.iloc[:,:61].describe()

,CH_1,CH_2,CH_3,CH_4,CH_5,CH_6,CH_7,CH_8,CH_9,CH_10,...,CH_52,CH_53,CH_54,CH_55,CH_56,CH_57,CH_58,CH_59,CH_60,CH_61
count,507025.000000,507025.000000,507025.000000,507025.000000,507025.000000,507025.000000,507025.000000,507025.000000,507025.000000,507025.000000,...,507025.000000,507025.000000,507025.000000,507025.000000,507025.000000,507025.000000,507025.000000,507025.000000,507025.000000,507025.000000
mean,211.260000,-107458.068479,-83945.053488,-75942.404354,-103164.841146,-60166.487843,-39980.218778,-12617.111662,44837.251064,12756.745575,...,-83487.328851,-70165.817717,-96149.080304,-14624.941839,-22444.241199,-28528.605265,-65102.316006,-32529.935054,-2038.068605,51706.430555
std,121.971379,998.419347,1446.320544,990.284365,916.029018,1056.326673,1209.745580,1934.952619,1493.769177,2381.103458,...,779.766874,725.456032,917.221820,1205.416564,1454.279490,606.348303,899.869603,799.827436,1033.234762,729.529380
min,0.000000,-108750.093750,-86397.375000,-78346.015625,-104687.718750,-62050.808594,-42287.375000,-16689.199219,0.000000,0.000000,...,-122897.617188,-103387.226562,-141396.031250,-25925.617188,-36307.609375,-42813.242188,-97516.468750,-50292.171875,-6704.064453,0.000000
25%,105.630000,-108199.273438,-85126.500000,-76481.500000,-103784.914062,-61214.636719,-41402.292969,-14086.449219,43337.929688,11399.105469,...,-84008.757812,-70570.437500,-96803.460938,-15245.446289,-23603.906250,-28908.287109,-65684.984375,-33270.269531,-2742.752930,51407.484375
50%,211.260000,-107472.203125,-83737.554688,-75745.953125,-103064.968750,-59683.628906,-39513.355469,-11936.872070,45398.285156,13714.065430,...,-83300.460938,-70017.398438,-95982.343750,-13987.609375,-21943.718750,-28422.324219,-64922.500000,-32520.023438,-1627.766846,51896.367188
75%,316.890000,-106685.882812,-83194.710938,-75178.718750,-102528.617188,-59306.000000,-38937.097656,-11062.363281,46067.828125,14422.068359,...,-83037.617188,-69615.078125,-95500.523438,-13768.730469,-21134.582031,-28012.712891,-64333.898438,-31740.902344,-1153.763184,52262.371094
max,422.520000,56377.390625,56676.359375,56246.585938,56438.781250,56112.914062,56406.453125,56493.101562,56266.710938,56637.281250,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,72322.281250
